In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from ape import networks, Contract
import pandas as pd

from helpers.coingecko import get_prices
from helpers.subgraph_endpoints import *
from helpers.aura import get_gravi_in_balancer_pool, weekly_emissions_after_fee, aura_mint_ratio, BALANCER_EMISSIONS
from helpers.vp_info import get_council_vp_fee, get_voter_vp
from helpers.bribe_info import MAX_BRIBE, GAUGE_CAP
from helpers.formatter import pct_format, dollar_format
from helpers.visuals import incentive_visuals, pool_capture_visuals, pnl_visuals, VISUAL_LABELS

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [3]:
bal_price, aura_price,  badger_price, _ = get_prices()
data = [["${:.2f}".format(badger_price),  "${:.2f}".format(bal_price), "${:.2f}".format(aura_price)]]
headers = ["Badger price", "Balancer price", "Aura price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,Badger price,Balancer price,Aura price
0,$2.99,$6.33,$3.33


In [4]:
# ping a current block, choose latest block at running time
block_current_proposal = networks.active_provider.get_block("latest").number
last_proposal_id = "0xcd6d5563958694cef110212fc9d3e270cbffeeae62ed44483e496ac543c6de2e"

# check active voter in past proposal as ref
response_last_proposal = get_proposal_info(last_proposal_id)

aura_voting_actively_last_round = response_last_proposal["scores_total"]

# TODO: pendant of finding out precisely active voters via onchain, fallback api?
proactive_vebal_voters = 9429222.35
cost_per_vote_vebal = (BALANCER_EMISSIONS * bal_price) / proactive_vebal_voters

data = [[block_current_proposal,  f'{aura_voting_actively_last_round:.0f}', f'{proactive_vebal_voters:.0f}', dollar_format(cost_per_vote_vebal)]]
headers = ["Proposal block height", "Past Round Active vlAURA votes", "Past Round Active veBAL votes", "Cost per Vote Last Round"]
df_llama = pd.DataFrame(data, columns=headers)
df_llama


,Proposal block height,Past Round Active vlAURA votes,Past Round Active veBAL votes,Cost per Vote Last Round
0,16841221,12065601,9429222,$0.0973


In [5]:
# emissions ecosystem
aura = Contract('0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF')
aura_mint_ratio = aura_mint_ratio(aura, block_current_proposal)
weekly_emissions_after_fee = weekly_emissions_after_fee(aura_mint_ratio, bal_price, aura_price)
biweekly_emissions = weekly_emissions_after_fee * 2
data = [[aura_mint_ratio, dollar_format(biweekly_emissions)]]
headers = ["Aura Mint Balancer Ratio", "Bi-weekly Emissions"]
df_emissions = pd.DataFrame(data, columns=headers)
df_emissions

,Aura Mint Balancer Ratio,Bi-weekly Emissions
0,3.472884,$3892097.8659


In [6]:
vlAURA = Contract("0x3Fa73f1E5d8A792C80F426fc8F84FBF7Ce9bBCAC")
vlAURA_ts = vlAURA.totalSupply(block_identifier=block_current_proposal)/1e18
council_fee = get_council_vp_fee(vlAURA, block_current_proposal)

# digg pool
balancer_vault = Contract("0xBA12222222228d8Ba445958a75a0704d566BF2C8")
gravi_in_digg_pool = get_gravi_in_balancer_pool(balancer_vault, block_current_proposal)
total_gravi_vp_for_digg = gravi_in_digg_pool

treasury_votes = get_voter_vp(vlAURA, block_current_proposal)
total_vp_badgerdao = treasury_votes + council_fee

pct_controlled_aura_by_badger_treasury = treasury_votes / vlAURA_ts
pct_controlled_aura_by_badger_treasury_active = treasury_votes / aura_voting_actively_last_round
pct_controlled_aura_by_badger = total_vp_badgerdao/ vlAURA_ts
pct_controlled_aura_by_badger_active = total_vp_badgerdao/ aura_voting_actively_last_round
pct_controlled_aura_by_council_active = total_gravi_vp_for_digg / aura_voting_actively_last_round
data = [[treasury_votes, council_fee, total_vp_badgerdao, gravi_in_digg_pool, pct_format(pct_controlled_aura_by_badger), pct_format(pct_controlled_aura_by_badger_active)]]
headers = ["Treasury VP", "Council Fee", "Total Badger VP", "Gravi in Digg pool", "Badger Aura Controlled - everyone Votes", "Badger Aura Controlled - proactive voters"]
df_vp = pd.DataFrame(data, columns=headers)
df_vp

,Treasury VP,Council Fee,Total Badger VP,Gravi in Digg pool,Badger Aura Controlled - everyone Votes,Badger Aura Controlled - proactive voters
0,348688.958034,52449.465144,401138.423177,206783.852373,3.011%,3.325%


In [7]:
vebal = Contract("0xC128a9954e6c874eA3d62ce62B468bA073093F25")
vebal_ts = vebal.totalSupplyAt(block_current_proposal) / 1e18
vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", block_current_proposal)/1e18
pct_aura_vebal_controlled = vebal_aura_controlled/vebal_ts
pct_badger_vebal_controlled = pct_controlled_aura_by_badger*pct_aura_vebal_controlled
pct_badger_vebal_controlled_active = pct_controlled_aura_by_badger_active*pct_aura_vebal_controlled
pct_council_vebal_controlled_active = pct_controlled_aura_by_council_active * pct_aura_vebal_controlled
data = [[pct_format(pct_aura_vebal_controlled), pct_format(pct_badger_vebal_controlled), pct_format(pct_badger_vebal_controlled_active)]]
headers = ["Aura veBAL controlled", "Badger veBAL controlled - everyone votes", "Badger veBAL controlled - proactive voters"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Aura veBAL controlled,Badger veBAL controlled - everyone votes,Badger veBAL controlled - proactive voters
0,27.259%,0.821%,0.906%


In [8]:
# NOTE: block breakdown for badger/wbtc max cap
vebal_controlled = vebal_ts * pct_badger_vebal_controlled_active
vebal_max_cap = vebal_ts * (GAUGE_CAP)
vebal_required = vebal_max_cap - vebal_controlled
cost_max_cap = vebal_required * cost_per_vote_vebal
badger_to_bribe = cost_max_cap / badger_price
remaining_bribe = MAX_BRIBE - badger_to_bribe
data = [[vebal_max_cap, vebal_controlled, dollar_format(cost_max_cap), badger_to_bribe]]
headers = ["veBAL badger/wbtc pool Max Cap", "veBAL BADGER controlled", "$ Cost of bribe", "Badger Bribe Max Cap"]
df_bribes_badger_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_pool

,veBAL badger/wbtc pool Max Cap,veBAL BADGER controlled,$ Cost of bribe,Badger Bribe Max Cap
0,203657.921622,92283.817731,$10841.2674,3625.841954


In [9]:
# treasury expected earnings bi-weekly expectation in BADGER/RETH pool
tvl_rethbadger = get_tvl_balancer_badgerreth_pool()
badgereth_pool = Contract("0x1ee442b5326009Bb18F2F472d3e0061513d1A0fF")
pool_badgereth_supply = badgereth_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0xaad4ee162dbc9c25cca26ba4340b36e3ef7c1a80")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_rethbadger * (vault_rewards_contract_balance/pool_badgereth_supply)
pool_capture_badger_reth = vault_owned_tcl / tvl_rethbadger
# NOTE: since self-incentive parties are voting also for this guage its reaching its nearly its max cap
vebal_pct_weight_bought = .1
pool_earnings_badger_reth = biweekly_emissions * vebal_pct_weight_bought

In [10]:
# treasury expected earnings bi-weekly expectation in BADGER/WBTC pool
tvl_badger = get_tvl_balancer_badger_pool()
badger_pool = Contract("0xb460DAa847c45f1C4a41cb05BFB3b51c92e41B36")
pool_supply = badger_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0x4efc8ded860bc472fa8d938dc3fd4946bc1a0a18")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_badger * (vault_rewards_contract_balance/pool_supply)
pool_capture_badger_wbtc = vault_owned_tcl / tvl_badger
pool_earnings_badger_wbtc = biweekly_emissions * GAUGE_CAP

In [11]:
# Incentive split visual
chart = incentive_visuals(VISUAL_LABELS, [badger_to_bribe, remaining_bribe])
chart.properties(height=95)

alt.LayerChart(...)

In [12]:
# Pool capture visual
chart = pool_capture_visuals(VISUAL_LABELS, [pool_capture_badger_wbtc, pool_capture_badger_reth])
chart.properties(height=95)

alt.LayerChart(...)

In [13]:
# Incentives ROI visual
chart = pnl_visuals(
    VISUAL_LABELS,
    [badger_to_bribe * badger_price, remaining_bribe * badger_price],
    [
        pool_earnings_badger_wbtc * pool_capture_badger_wbtc,
        pool_earnings_badger_reth * pool_capture_badger_reth,
    ],
)
chart.properties(height=100)

alt.Chart(...)